In [7]:
import pandas as pd
import numpy as np
#import osmnx as ox
import networkx as nx
import time

In [8]:
# Load data
df = pd.read_csv("bike_data_new.csv")

In [9]:
df.head()

,trip_id,started_at,ended_at,start_lat,start_lng,end_lat,end_lng
0,1,01-02-2023 00:03,01-02-2023 00:13,38.905470,-77.002130,38.906299,-76.983221
1,2,01-02-2023 00:06,01-02-2023 00:19,38.907693,-77.071512,38.909858,-77.038566
2,3,01-02-2023 00:07,01-02-2023 00:14,38.915544,-77.038252,38.916137,-77.022003
3,4,01-02-2023 00:07,01-02-2023 00:26,38.909701,-77.085646,38.935839,-77.106913
4,5,01-02-2023 00:10,01-02-2023 00:20,38.905509,-77.065246,38.898302,-77.046913


In [10]:
# Remove trips of duration 0 minutes
df = df[df['started_at'] != df['ended_at']]

# Calculate duration in minutes
df['duration'] = (pd.to_datetime(df['ended_at']) - pd.to_datetime(df['started_at'])).dt.seconds / 60

# Print the required values
print('Maximum duration of the trip:', df['duration'].max(), 'minutes')
print('Minimum duration of the trip:', df['duration'].min(), 'minutes')
print('Total number of trips corresponding to the minimum duration:', len(df[df['duration'] == df['duration'].min()]))
print('Percentage of total circular trips:', round((len(df[df['start_lat'] == df['end_lat']]) / len(df)) * 100, 2), '%')

Maximum duration of the trip: 518.0 minutes
Minimum duration of the trip: 1.0 minutes
Total number of trips corresponding to the minimum duration: 89
Percentage of total circular trips: 2.95 %


In [11]:
# Convert start time to datetime
df['started_at'] = pd.to_datetime(df['started_at'])

# Filter the dataframe to include only trips starting between 06:00 AM and 06:00 PM
df = df[(df['started_at'].dt.hour >= 6) & (df['started_at'].dt.hour < 18)]

# Sort the dataframe by start time
df = df.sort_values('started_at')

# Create a dictionary of end locations and their corresponding start times
end_locations = {}
for index, row in df.iterrows():
    end_loc = (row['end_lat'], row['end_lng'])
    start_time = row['started_at']
    if end_loc in end_locations:
        end_locations[end_loc].append(start_time)
    else:
        end_locations[end_loc] = [start_time]

# Count the number of feasible pairs of trips
feasible_pairs = 0
for index, row in df.iterrows():
    end_loc = (row['end_lat'], row['end_lng'])
    start_time = row['started_at']
    if end_loc in end_locations:
        for time in end_locations[end_loc]:
            if time > start_time:
                feasible_pairs += 1

# Print the total feasible pairs of trips and runtime
print('Total feasible pairs of trips:', feasible_pairs)


Total feasible pairs of trips: 54390


In [16]:
import osmnx as ox
import networkx as nx

graph = ox.graph_from_place('City Name, Country Name', network_type='walk')
nx.write_gpickle(graph, 'graph.pkl')

ModuleNotFoundError: No module named 'osmnx'